In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/wsdm-cup-multilingual-chatbot-arena/sample_submission.csv
/kaggle/input/wsdm-cup-multilingual-chatbot-arena/train.parquet
/kaggle/input/wsdm-cup-multilingual-chatbot-arena/test.parquet
/kaggle/input/bert-model/bert_base_uncased_model/config.json
/kaggle/input/bert-model/bert_base_uncased_model/model.safetensors
/kaggle/input/bert-token/bert_base_token/tokenizer.json
/kaggle/input/bert-token/bert_base_token/tokenizer_config.json
/kaggle/input/bert-token/bert_base_token/special_tokens_map.json
/kaggle/input/bert-token/bert_base_token/vocab.txt
/kaggle/input/token-ml/tokenzer_ml_bert/tokenizer.json
/kaggle/input/token-ml/tokenzer_ml_bert/tokenizer_config.json
/kaggle/input/token-ml/tokenzer_ml_bert/special_tokens_map.json
/kaggle/input/token-ml/tokenzer_ml_bert/vocab.txt
/kaggle/input/model-ml/Model_ml_bert/config.json
/kaggle/input/model-ml/Model_ml_bert/model.safetensors


In [2]:
ss = pd.read_csv("/kaggle/input/wsdm-cup-multilingual-chatbot-arena/sample_submission.csv")
ss.head()

,id,winner
0,327228,model_a
1,1139415,model_a
2,1235630,model_a


In [3]:
from datasets import Dataset

# Load the datasets from parquet files
train_df = pd.read_parquet('/kaggle/input/wsdm-cup-multilingual-chatbot-arena/train.parquet')
test_df = pd.read_parquet('/kaggle/input/wsdm-cup-multilingual-chatbot-arena/test.parquet')

# Prepare training dataset for classification
train_samples = []
for _, row in train_df.iterrows():
    train_samples.append({
        'prompt': row['prompt'],
        'response': row['response_a'],
        'winner': 0 if row['winner'] == 'model_a' else 1
    })
    train_samples.append({
        'prompt': row['prompt'],
        'response': row['response_b'],
        'winner': 1 if row['winner'] == 'model_b' else 0
    })

# Convert to a Hugging Face Dataset
train_dataset = Dataset.from_dict({
    'prompt': [sample['prompt'] for sample in train_samples],
    'response': [sample['response'] for sample in train_samples],
    'winner': [sample['winner'] for sample in train_samples],
})

In [4]:
train_df.head()

,id,prompt,response_a,response_b,winner,model_a,model_b,language
0,00007cff95d7f7974642a785aca248b0f26e60d3312fac...,vieš po Slovensky?,"Áno, hovorím po slovensky. Ako vám môžem pomôcť?","Áno, veď som tu! Môžem ti pomôcť s otázkami al...",model_a,o1-preview,reka-core-20240904,Slovak
1,00010ed04b536f56ebe43eef1100c13906abea12bf9855...,You will be given a piece of news. Analyze it ...,Let's break down the news and analyze it accor...,"```json\n{\n ""contains_orgs"": true,\n ""orgs""...",model_a,gemma-2-27b-it,gemini-1.5-flash-002,Russian
2,0003800d510e38803efba5ceaec122bc66408fe367b0be...,"Dört basamaklı, rakamları birbirinden ve sıfır...","Bu soruyu çözmek için, verilen koşulları adım ...",Bu problemi adım adım çözelim:\n\n1) ABCD - DC...,model_a,gpt-4-0125-preview,claude-3-5-sonnet-20240620,Turkish
3,00072026c68f5418ef2da238394e418ce72a534b9b22d5...,"현재 추천된 탑 3 종목인 Cabaletta Bio (CABA), Rocket Ph...","죄송하지만 저는 금융 조언을 제공할 수 없습니다. 저는 AI 모델이며, 투자 결정에...",현재 추천된 탑 3 종목에 순위를 매기기 위해서는 여러 가지 요소들을 고려해야 합니...,model_b,gemma-2-2b-it,llama-3.1-nemotron-70b-instruct,English
4,0007ce7cf6bc1b5a8f8a4669b854fb12030863c970d9dc...,Please be boring,"Alright, I'll be as boring as possible.\n\nTod...","Understood. Here is a straightforward, unadorn...",model_a,reka-flash-20240722,grok-2-2024-08-13,English


In [5]:
train_df["language"].unique()

array(['Slovak', 'Russian', 'Turkish', 'English', 'Korean', 'Swedish',
       'Vietnamese', 'Finnish', 'unknown', 'Chinese', 'Hungarian',
       'Spanish', 'Polish', 'Japanese', 'Dutch', 'German', 'Yiddish',
       'Italian', 'French', 'Indonesian', 'Persian', 'Portuguese',
       'Catalan', 'Czech', 'Ukrainian', 'Arabic', 'Bislama', 'Bulgarian',
       'Romanian', 'Armenian', 'Xhosa', 'Yoruba', 'Thai', 'Serbian',
       'Slovenian', 'Tamil', 'Sanskrit', 'xx', 'Malay', 'Swahili',
       'Irish', 'Lithuanian', 'Waray', 'Kyrgyz', 'Kalaallisut', 'Latin',
       'Nauru', 'Corsican', 'Khasi', 'Norwegian', 'Hebrew', 'Faroese',
       'Azerbaijani', 'Danish', 'Abkhazian', 'Oromo', 'Ganda', 'Zhuang',
       'Estonian', 'Basque', 'Uzbek', 'Pashto', 'Afrikaans', 'Scots',
       'Macedonian', 'Greek', 'Luxembourgish', 'Latvian', 'Wolof',
       'Croatian', 'Esperanto', 'Galician', 'Javanese', 'Bosnian',
       'Tongan', 'Belarusian', 'Bangla', 'Romansh', 'Sundanese',
       'Tagalog', 'Southern S

In [6]:
test_df.head()

,id,prompt,response_a,response_b,scored
0,327228,Caso Clínico: Un hombre de 70 años con anteced...,**Diagnóstico Diferencial de Anemia en Pacient...,"Basándonos en el caso clínico presentado, pode...",False
1,1139415,Peel Company received a cash dividend from a ...,The correct answer is **(a) No No**. Here's ...,The correct answer is **(a) No No**. Here's wh...,False
2,1235630,Há um grave problema com o relógio da torre da...,Dois problemas interessantes!\n\n**Problema 1:...,Vamos resolver os dois problemas em sequência....,False


In [7]:
# Use a subset of the dataset for quick testing
#train_dataset = train_dataset.select(range(2000))  # Select first 1000 examples

In [8]:
train_dataset

Dataset({
    features: ['prompt', 'response', 'winner'],
    num_rows: 96878
})

In [9]:
from transformers import AutoModelForSequenceClassification
import torch.nn.utils.weight_norm
import torch.nn as nn

class WeightNormModel(AutoModelForSequenceClassification):

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        # Applying weight normalization on the classifier layer
        self.classifier = torch.nn.utils.weight_norm(self.classifier)

# Load your model with weight normalization
#model = WeightNormModel.from_pretrained(
    #"/kaggle/input/bert-model/bert_base_uncased_model",
    #num_labels=2,
#)

model = AutoModelForSequenceClassification.from_pretrained(
    "/kaggle/input/model-ml/Model_ml_bert",  # You can choose other model variants as well
    num_labels=2,
    id2label={0: "model_a", 1: "model_b"},  # Labels corresponding to models
    label2id={"model_a": 0, "model_b": 1},
)

# Freeze all parameters in the base model
for param in model.base_model.parameters():
    param.requires_grad = False

# Unfreeze the last 3 layers of the transformer
num_layers = len(model.base_model.encoder.layer)
for layer in model.base_model.encoder.layer[num_layers-3:]:
    for param in layer.parameters():
        param.requires_grad = True

In [10]:
total_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Number of parameters that require gradients: {total_parameters}")

Number of parameters that require gradients: 21265154


In [11]:
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding
from transformers import AutoTokenizer
from datasets import DatasetDict
import torch
import random

# Set random seed for reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

# Ensure deterministic behavior
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


# Tokenizer
tokenizer = AutoTokenizer.from_pretrained("/kaggle/input/token-ml/tokenzer_ml_bert")

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples['prompt'], examples['response'], return_tensors='pt', padding=True, truncation=True)

# Tokenize the dataset
tokenized_train = train_dataset.map(
    lambda examples: {
        **tokenize_function(examples),
        "labels": examples["winner"],  # Replace 'label' with the actual column name in your dataset
    },
    batched=True,
)

# Split the tokenized_train dataset into training and evaluation datasets
split_datasets = tokenized_train.train_test_split(test_size=0.2, seed=42)  # Adjust test_size as needed
tokenized_train = split_datasets["train"]
tokenized_eval = split_datasets["test"]

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}


# Prepare training arguments
training_args = TrainingArguments(
    output_dir="./model_output",
    learning_rate=1e-4,
    per_device_train_batch_size=150,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    logging_dir="./logs",
    logging_steps=10,
    report_to="none",  # Disable logging to WandB
)

# Data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,  # Provide a validation dataset
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,  # Add compute_metrics
)

# Train the model
trainer.train()

Map:   0%|          | 0/96878 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy
1,0.693700,0.693084,0.507690
2,0.691500,0.692130,0.513625
3,0.688400,0.688886,0.535456
4,0.657400,0.685461,0.567971
5,0.600900,0.705334,0.577364


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

TrainOutput(global_step=1295, training_loss=0.6707135671814436, metrics={'train_runtime': 12180.9963, 'train_samples_per_second': 31.813, 'train_steps_per_second': 0.106, 'total_flos': 1.019581650625536e+17, 'train_loss': 0.6707135671814436, 'epoch': 5.0})

In [12]:
# Prepare test dataset for predictions
test_samples = []
for _, row in test_df.iterrows():
    test_samples.append({
        'id': row['id'],
        'prompt': row['prompt'],
        'response': row['response_a'],
    })
    test_samples.append({
        'id': row['id'],
        'prompt': row['prompt'],
        'response': row['response_b'],
    })

# Convert to a Hugging Face Dataset
test_dataset = Dataset.from_dict({
    'id': [sample['id'] for sample in test_samples],
    'prompt': [sample['prompt'] for sample in test_samples],
    'response': [sample['response'] for sample in test_samples],
})

In [13]:
# Tokenize the test dataset
tokenized_test = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/6 [00:00<?, ? examples/s]

In [14]:
# Step 3: Make predictions
predictions = trainer.predict(tokenized_test)
predicted_labels = np.argmax(predictions.predictions, axis=1)

In [15]:
# Prepare submission DataFrame
submission_data = []
for i in range(len(tokenized_test)):
    # The ID from the test dataset
    sample_id = tokenized_test['id'][i]  
    # Determining the winner based on the predicted label
    winner = 'model_a' if predicted_labels[i] == 0 else 'model_b'
    
    # Append to submission data
    submission_data.append({
        'id': sample_id,
        'winner': winner,
    })

# Create DataFrame
submission_df = pd.DataFrame(submission_data)

# Group by 'id' and take the first predicted winner for each unique id
submission_df = submission_df.groupby('id', as_index=False).first()

# Save as CSV
submission_df.to_csv('submission.csv', index=False)

In [16]:
submission_df

,id,winner
0,327228,model_b
1,1139415,model_b
2,1235630,model_a
